# GraphGuard

***Locate and find Classes in Apks with updated Obfuscation Mapping***


Processing Steps:
1. String Matcher (Finding Classes and Methods
  * Counting Strings used in Classes and Methods and try to find exact matching counter.
  * Find Classes by identifying Strings used only in this single Class.
2. Structure Matcher (Finding Classes)
  * Modifiers of class
  * Modifiers, Parameters, Parameter Types and Return Types of Methods
  * Number and Types of Fields.
3. Method Matcher (Find Methods from matching Classes)
  * Modifiers
  * Return Type, Parameter Types
  * Bytecode Length
  * References to and from


***If you use Akrolyb, please have a look at [Akrolyb Interoptability](#Interoptability-with-Akrolyb)***

In [1]:
%matplotlib notebook


from IPython.core.display import display, HTML
display(HTML("<style>div.output_area pre {white-space: pre;}</style>"))

In [2]:
from formats import *
from decs import *

from matching import matcher, strings, structures, methods
import generate

from start import process_files

# Loading Androguard

The following code loads the files and starts Androguard

It should support multiprocessing, however the Pipe communication seems to break when transmitting the processed Androguard Objects. I suspect the Object is simply too big for Pickle to serialize or another component in the transmitting chain.

In [3]:
AG_SESSION_FILE = "./Androguard.ag"
MULTIPROCESS_FILES = False  # Currently not working due to serialization issues


# Matching Rules
strings.MAX_USAGE_COUNT_STR = 20
strings.UNIQUE_STRINGS_MAJORITY = 2 / 3

methods.MIN_MATCH_POINTS = 2



# APK Files to load
file_paths = (
    "../../../Downloads/com.snapchat.android_10.85.5.74-2067_minAPI19(arm64-v8a)(nodpi)_apkmirror.com.apk",
    "../../../Downloads/com.snapchat.android_10.86.5.61-2069_minAPI19(arm64-v8a)(nodpi)_apkmirror.com.apk"
)

In [4]:
(a, d, dx), (a2, d2, dx2) = process_files(file_paths, MULTIPROCESS_FILES)

Loading Session from Apk at ../../../Downloads/com.snapchat.android_10.85.5.74-2067_minAPI19(arm64-v8a)(nodpi)_apkmirror.com.apk
Loading Session from Apk at ../../../Downloads/com.snapchat.android_10.86.5.61-2069_minAPI19(arm64-v8a)(nodpi)_apkmirror.com.apk


# List of Methods

Defining the list of methods to find (obviously requires full class names)

In [5]:
decs_to_find = (
    MethodDec("rD5", "a", "rD5", "qD5"),
    MethodDec("MSg", "j0", "SGd"),
    MethodDec("xke", "g", "PV4"),
    MethodDec("GIb", "<init>", skip_params=True)
)

# Processing and Matching

Loading the accumulator, an object that manages all the possible candidates that are matched by the different Matchers, and extracts the matching candidates. It also performs Inner joins on previous candidates to find the exact (or optimal) match.

In [6]:
accumulator = matcher.Accumulator()

Resolving the previously defined MethodDecs. If this fails, the MethodDecs are wrong and contain an error. Make sure the method specified with the MethodDec exists.

In [7]:
resolved_classes = resolve_classes(dx, decs_to_find)
resolved_methods = resolve_methods(decs_to_find, resolved_classes)
decs_ma = dict(zip(decs_to_find, resolved_methods))

print("Resolved all Classes and Methods")
if False:
    print("", *map(pretty_format_ma,resolved_methods), sep="\n* ")

# Arguments to provider to Matchers
args = (dx, dx2, resolved_classes, decs_ma)

Resolved all Classes and Methods


## String Matcher

### Exact Counter Match

Extracts Strings used either in the given methods directly or in the classes the methods define for both, the old version and the new version. It then compares the Counters for classes and methods and tries to find exact matches between the Counter Objects.

In [8]:
string_matcher = strings.StringMatcher(*args, accumulator.get_unmatched_ms(decs_to_find))
candidates_cs, candidates_ms = string_matcher.compare_counters()

accumulator.add_candidates(candidates_cs, candidates_ms)

+ Found single candidate for matching Class. Considering it a match! 
	LrD5; -> LQE5;
+ Found single candidate for matching Class. Considering it a match! 
	Lxke; -> Lvre;
+ Found single candidate for matching Class. Considering it a match! 
	LMSg; -> LV0h;
+ Found single candidate for matching Method. Considering it a match 
	rD5#a(rD5, qD5) -> QE5#a(QE5, PE5)
+ Found single candidate for matching Method. Considering it a match 
	MSg#j0(SGd) -> V0h#j0(IMd)
Could resolve 3 new Classes, 2 new Methods


### Unique Strings

Gather all Strings that are used only in a single class ("Unique Strings") that we still need to match. Then try to find the matching class by only searching for the Unique Strings.

In [9]:
candidates_cs = string_matcher.compare_unique_strings(accumulator.get_unmatched_cs(decs_to_find))

accumulator.add_candidates(candidates_cs)

Could resolve 0 new Classes, 0 new Methods


## Structure Matcher

Iterating through every single class and checks for each unmatched class if both have a similar "Profile":
* Number of Methods and Fields
* Types of Fields and Descriptors of Methods

In [10]:
structure_matcher = structures.StructureMatcher(*args, accumulator.get_unmatched_ms(decs_to_find))
candidates_cs = structure_matcher.get_exact_structure_matches(accumulator.get_unmatched_cs(decs_to_find))

accumulator.add_candidates(candidates_cs)

+ Found single candidate for matching Class. Considering it a match! 
	LGIb; -> LFOb;
Could resolve 1 new Classes, 0 new Methods


## Method Matcher

Uses different weighted criteria to get exact or optimal matches. The criteria are:
* Modifiers
* Return Type and Parameter Types
* Length of Bytecode
* References to and from

In [11]:
method_matcher = methods.MethodMatcher(*args, accumulator.get_unmatched_ms(decs_to_find))

print("Exact Matching")
print()

# Exact Matches
candidates_ms = method_matcher.try_resolve_ms(accumulator.get_unmatched_cs(decs_to_find), accumulator.matching_cs, True)
accumulator.add_candidates(candidates_ms=candidates_ms)

print()
print("Non-Exact Matching")
print()

# Non-Exact Matches by using weights on the criteria
candidates_ms = method_matcher.try_resolve_ms(accumulator.get_unmatched_cs(decs_to_find), accumulator.matching_cs, False)
accumulator.add_candidates(candidates_ms=candidates_ms)

Exact Matching

+ Found single candidate for matching Method. Considering it a match 
	GIb#<init>(skip.params) -> FOb#<init>(java.lang.String, boolean, java.lang.String, java.lang.String, java.lang.String, java.lang.Long, Xt5, long, boolean, java.lang.Long, long)
Could resolve 0 new Classes, 1 new Methods

Non-Exact Matching

+ Found single candidate for matching Method. Considering it a match 
	xke#g(PV4) -> vre#g(pX4)
+ Found single candidate for matching Method. Considering it a match 
	GIb#<init>(skip.params) -> FOb#<init>(java.lang.String, boolean, java.lang.String, java.lang.String, java.lang.String, java.lang.Long, Xt5, long, boolean, java.lang.Long, long)
Could resolve 0 new Classes, 1 new Methods


# Results
Display summary and matching pairs.

Output to compatible MethodDec input to update from auto-updated values.

In [12]:
print(len(accumulator.matching_cs), "/", len(resolved_classes), "Classes were matched")
print(len(accumulator.matching_ms), "/", len(decs_to_find), "Methods were matched")

print()
print("Matching Classes:")
for c1, c2 in accumulator.matching_cs.items():
    print("•", pretty_format_class(c1), "->", pretty_format_class(c2))

print()
print("Matching Methods: ")
for m, ma in accumulator.matching_ms.items():
    print("•", m.pretty_format(), "->", pretty_format_ma(ma))

4 / 4 Classes were matched
4 / 4 Methods were matched

Matching Classes:
• rD5 -> QE5
• xke -> vre
• MSg -> V0h
• GIb -> FOb

Matching Methods: 
• rD5#a(rD5, qD5) -> QE5#a(QE5, PE5)
• MSg#j0(SGd) -> V0h#j0(IMd)
• GIb#<init>(skip.params) -> FOb#<init>(java.lang.String, boolean, java.lang.String, java.lang.String, java.lang.String, java.lang.Long, Xt5, long, boolean, java.lang.Long, long)
• xke#g(PV4) -> vre#g(pX4)


In [13]:
generate.generate_decs(decs_to_find, accumulator.matching_ms)

decs_to_find = (
	MethodDec('QE5', 'a', 'QE5', 'PE5'),
	MethodDec('V0h', 'j0', 'IMd'),
	MethodDec('vre', 'g', 'pX4'),
	MethodDec('FOb', '<init>', 'java.lang.String', 'boolean', 'java.lang.String', 'java.lang.String', 'java.lang.String', 'java.lang.Long', 'Xt5', 'long', 'boolean', 'java.lang.Long', 'long')
)


# Interoptability with Akrolyb

The following Code snippet allows to "extract" MethodDecs from MemberDeclarations in Akrolyb. This automates the process of providing GraphGuard the hooks it should find. Just paste the output (tuple of MethodDecs) in the corresponding section in this Notebook.

## Extract

It is not in Python, since it would require a Kotlin Parser and evaluating imports, variables etc. Just executing the Constructors in Kotlin, then getting the values is much easier than static analysis. 

The `main` function can (and should) be run statically (locally on the machine and not on your Android) to build the list of `MethodDec`s that GraphGuard is supposed to match in an updated build of the target application. Adjust the MemberDeclarations Class to the Class where you declared the `MemberDeclaration`s.

Note: This code does obviously not know if you left out the parameters on purpose (a lot of parameter types, or multiple constructors...). Please modify the results for the Methods concerned by using the optional `skip_params=True` argument for `MethodDec`.

```kotlin
fun main() {
    fun MemberDec.formatToGraphGuard() = buildString {
        append("MethodDec('")
        append(classDec.className)
        append("', '")
        append(if (this@formatToGraphGuard is MethodDec) methodName else "<init>")
        append("'")
        for (param in params) {
            append(", '")
            if (param is String)
                append(param)
            else if (param is ClassDec)
                append(param.className)
            else if (param is Class<*>)
                append(param.name)
            else error("Illegal Type for param")
            append("'")
        }
        append(")")
    }
    
    println("decs_to_find = (")
    val strs = MemberDeclarations::class.java.declaredFields.mapNotNull {
        it.isAccessible = true
        val x = it.get(MemberDeclarations)
        if (x !is MemberDec) return@mapNotNull null

        x.formatToGraphGuard()
    }.joinToString(separator = ",\n\t", prefix = "\t")
    println(strs)
    println(")")
}
```